In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import sys
import pandas as pd
import numpy as np
import requests
from spotify_dl import spotify_dl
from pathlib import Path
import time
import os
from dotenv import load_dotenv  # changed magic command to explicit load
import librosa
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import pairwise
from sklearn.model_selection import train_test_split
from typing import List
from flask import Flask, redirect, request
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.callbacks import EarlyStopping
from skimage.transform import resize


pd.set_option("display.max_rows", None)  # pandas dataframe formatting options
pd.set_option("display.max_columns", None)


custom_env_path = "../../../brainstation_capstone_cfg.env"  # environment variables file

In [2]:
# Spotify Developer Credentials
load_dotenv(dotenv_path=custom_env_path)
CLIENT_ID = os.environ.get("SPOTIPY_CLIENT_ID")
# client ID from app
CLIENT_SECRET = os.environ.get("SPOTIPY_CLIENT_SECRET")
# client secret from app
REDIRECT_URI = os.environ.get("REDIRECT_URI")
# redirect URI - the URI used here matches the one used within the app
SCOPE = "{} {}".format(os.environ.get("SCOPE_PUBLIC"), os.environ.get("SCOPE_PRIVATE"))
# formatted the scope this way to allow for custom configurations in the future
USERNAME = os.environ.get("SPOTIFY_USERNAME")
# Spotify username

In [11]:
kaggle_df = pd.read_csv('../../data/SpotifyFeatures.csv')

In [4]:
normal_df = pd.read_parquet(
    "../../data/vectorized_mp3s/pairwise_complete_parquets/20230901_complete_pairwise_data.parquet"
)
normal_df.shape

(11578, 498)

In [5]:
cnn_df = pd.read_parquet(
    "../../data/vectorized_mp3s/cnn_complete_parquets/20230901_complete_cnn_data.parquet"
)
cnn_df.shape

(11578, 1000)

In [6]:
normal_similarity = pairwise.cosine_similarity(normal_df, dense_output=True)
normal_similarity

array([[1.        , 0.97973231, 0.99154322, ..., 0.98798471, 0.9864652 ,
        0.96577147],
       [0.97973231, 1.        , 0.98947364, ..., 0.98547464, 0.99003342,
        0.98766516],
       [0.99154322, 0.98947364, 1.        , ..., 0.99357991, 0.99600081,
        0.98262735],
       ...,
       [0.98798471, 0.98547464, 0.99357991, ..., 1.        , 0.99601647,
        0.98547392],
       [0.9864652 , 0.99003342, 0.99600081, ..., 0.99601647, 1.        ,
        0.9892527 ],
       [0.96577147, 0.98766516, 0.98262735, ..., 0.98547392, 0.9892527 ,
        1.        ]])

In [7]:
normal_similarity_sorted = np.argsort(normal_similarity)[:, ::-1]

In [9]:
vec = list(normal_similarity_sorted[7085, 0:11])
normal_similarity[7085, vec]  # similarity for first match

array([1.        , 0.99678356, 0.99594064, 0.99573878, 0.99568158,
       0.99548641, 0.99543346, 0.99539609, 0.9953861 , 0.99531449,
       0.9949721 ])

In [ ]:
normal_recommendation_df = kaggle_df.copy(deep=True)
normal_recommendation_df["track_id"] = normal_recommendation_df["track_id"].astype(
    "category"
)
normal_recommendation_df[
    "track_id"
] = normal_recommendation_df.track_id.cat.set_categories(normal_recommendations[3])
# normal_recommendation_df[
#     "track_id"
# ] = normal_recommendation_df.track_id.cat.set_categories(normal_recommendations[3])

normal_track_recommendations = normal_recommendation_df.copy(deep=True).sort_values(
    ["track_id"]
)  # sorts heirarchy
# track_recommendations.head()
normal_track_recommendations = normal_track_recommendations.drop_duplicates(
    subset="track_id"
)
normal_track_recommendations.head(11)

In [12]:
unique_track_ids = pd.read_parquet('../../data/20230903_unique_track_ids_for_training.parquet')

In [13]:
track_ids = unique_track_ids.track_ids.unique().tolist()

In [14]:
training_mp3s_df = kaggle_df.loc[(kaggle_df.track_id.str.strip().isin(track_ids))].drop(columns = 'genre').drop_duplicates(subset='track_id')
# training_mp3s_df.to_parquet('../../data/20230905_training_mp3s.parquet')

In [17]:
training_mp3s_df.loc[training_mp3s_df.artist_name.str.lower().str.strip().str.contains('jessie ware')]

,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
86463,Jessie Ware,Til The End,3184gD1FUstNCMVZMtuQ9W,40,0.703,0.695,182827,0.398,0.239,F#,0.0783,-12.791,Minor,0.0652,145.958,4-Apr,0.578


#### CNN Similarity

In [62]:
cnn_similarity = pairwise.cosine_similarity(cnn_df, dense_output=True)
cnn_similarity

array([[0.9999999 , 0.96258837, 0.99975014, ..., 0.9996804 , 0.7478662 ,
        0.8212553 ],
       [0.96258837, 0.9999997 , 0.96280265, ..., 0.9655044 , 0.7904676 ,
        0.9046931 ],
       [0.99975014, 0.96280265, 0.99999994, ..., 0.9994428 , 0.7471412 ,
        0.81841034],
       ...,
       [0.9996804 , 0.9655044 , 0.9994428 , ..., 1.        , 0.75507843,
        0.83261484],
       [0.7478662 , 0.7904676 , 0.7471412 , ..., 0.75507843, 1.0000006 ,
        0.71857   ],
       [0.8212553 , 0.9046931 , 0.81841034, ..., 0.83261484, 0.71857   ,
        0.99999964]], dtype=float32)

In [63]:
cnn_similarity_sorted = np.argsort(cnn_similarity)[:, ::-1]

#### Check Normal Recommendations

In [66]:
normal_recommendation_df = kaggle_df.copy(deep=True)
dissimilar_recommendation_df = kaggle_df.copy(deep=True)
normal_recommendation_df["track_id"] = normal_recommendation_df["track_id"].astype(
    "category"
)
normal_recommendation_df[
    "track_id"
] = normal_recommendation_df.track_id.cat.set_categories(normal_recommendations[3])
# normal_recommendation_df[
#     "track_id"
# ] = normal_recommendation_df.track_id.cat.set_categories(normal_recommendations[3])

normal_track_recommendations = normal_recommendation_df.copy(deep=True).sort_values(
    ["track_id"]
)  # sorts heirarchy
# track_recommendations.head()
normal_track_recommendations = normal_track_recommendations.drop_duplicates(
    subset="track_id"
)
normal_track_recommendations.head(11)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
180270,Jazz,Green Assassin Dollar,Aware,2QbR9yeeX7XHznRZwsyh3F,34,0.8420,0.720,73500,0.268,0.958,A,0.1420,-12.323,Minor,0.1020,80.132,4-Apr,0.4040
180055,Jazz,BluntOne,Recogneyes,6zP7r08qkY9GIWJ7TxVV6E,35,0.7820,0.607,100043,0.354,0.855,B,0.1150,-8.413,Minor,0.1060,78.832,4-Apr,0.8310
146179,Jazz,BluntOne,Ghost Train Haze,56ir9LGNYpV3O4U5ShDvn9,50,0.5490,0.566,113468,0.125,0.543,B,0.1250,-9.770,Minor,0.1360,128.428,4-May,0.5240
61009,R&B,The Marías,Loverboy,3XCpEFU4uXsBq5WmVQQKC9,48,0.8080,0.651,73236,0.152,0.422,G,0.1170,-18.704,Minor,0.0832,131.088,4-Apr,0.0746
145515,Jazz,BluntOne,Loved Ones,43iOWRh6448rFtwBX6CAfZ,57,0.1630,0.811,99765,0.181,0.880,E,0.0861,-11.996,Minor,0.4460,38.498,4-Apr,0.4690
145808,Jazz,tusken.,mocean,0KFF0INOF9PT3GYWbVjfI4,56,0.7380,0.759,149318,0.225,0.843,B,0.1500,-13.165,Minor,0.1290,88.022,4-Apr,0.3850
25703,Electronic,Ozoyo,Where to Go (feat. Wun Two),0fDnP59qSG3dlo1j1z1fHl,34,0.1780,0.469,106667,0.372,0.909,A#,0.1480,-12.275,Major,0.3240,121.355,4-Mar,0.6670
145939,Jazz,FloFilz,Gelbstich,7niUZWARcEMEntcEOeiTH4,55,0.5430,0.740,117774,0.265,0.973,C#,0.1150,-11.693,Major,0.0461,82.040,4-Apr,0.4120
44120,Folk,Vancouver Sleep Clinic,Ayahuasca,6oVm5ocTB8B0rjO5dZk8Px,51,0.7340,0.463,508602,0.228,0.110,E,0.0691,-15.717,Minor,0.0338,148.003,4-Apr,0.0528
199078,Soundtrack,Ramin Djawadi,"To Vaes Dothrak - From The ""Game Of Thrones"" S...",6hJA1KQ8AK2r1mazUOQWGM,41,0.7970,0.465,88867,0.254,0.897,G#,0.0946,-24.967,Major,0.0351,160.108,4-Mar,0.1150


In [67]:
# kaggle_df.loc[kaggle_df.track_id.isin(dissimilar_recommendations[3])].sort_values(by ='track_id', key = dissimilar_recommendations[3])
kaggle_df.set_index("track_id").loc[dissimilar_recommendations[3]].reset_index(
    inplace=False
).drop_duplicates(subset="track_id")

,track_id,genre,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2QbR9yeeX7XHznRZwsyh3F,Jazz,Green Assassin Dollar,Aware,34,0.84200,0.720,73500,0.268,0.958000,A,0.1420,-12.323,Minor,0.1020,80.132,4-Apr,0.404
1,7wobmC9Xz4stYN1ZVLWgi5,Soul,Alicia Keys,Elaine Brown (Interlude),35,0.95500,0.673,50187,0.364,0.000000,A,0.2800,-12.828,Minor,0.9010,89.549,4-Jan,0.705
2,4oarDDCWsSnX2WOIRKuQo1,Blues,Jimi Hendrix,Up From The Skies,41,0.60300,0.659,177867,0.635,0.021400,A#,0.1720,-10.569,Minor,0.2020,144.965,4-Apr,0.413
3,11SY8jJkg1CtUgHx37qUlm,Dance,Miguel,Shockandawe,44,0.35200,0.766,150730,0.902,0.000000,C,0.4970,-7.067,Major,0.1040,109.032,4-Apr,0.470
5,2s9M78CDrQDjB2D827XXr4,Alternative,The Naked And Famous,Higher,48,0.00649,0.463,230747,0.766,0.000017,F#,0.5810,-5.383,Major,0.0511,184.075,4-Apr,0.173
9,5oICBRkaGVtLTkkKeXuSKF,Hip-Hop,Jeezy,Snow Season,49,0.09510,0.711,204907,0.569,0.000000,G#,0.0999,-7.832,Major,0.0560,101.993,4-Apr,0.155
11,0vybyrCk6ANFFmDTBWq74f,Alternative,Francis and the Lights,Cruise,38,0.10900,0.858,216161,0.332,0.065300,C,0.1160,-12.177,Major,0.0638,140.010,4-Apr,0.480
12,2C6FKqjI3CT4Q0YjOlAQAl,World,Michael W. Smith,Thy Word,26,0.19200,0.405,210373,0.412,0.756000,G,0.3000,-11.837,Major,0.0278,147.964,4-Apr,0.132
13,6cZ8T2c95NfRTrGEXqThIq,Country,Willie Nelson,Django and Jimmie,40,0.55600,0.547,173453,0.407,0.000110,D,0.3480,-12.214,Major,0.0381,129.030,4-Mar,0.329
14,2SNtwYtk9a4THKlERP0bMN,Alternative,Francis and the Lights,Morning,45,0.96900,0.657,97675,0.250,0.018600,D#,0.1080,-10.423,Major,0.0457,88.325,4-Apr,0.467


In [68]:
vec = list(normal_similarity_sorted[7085, -10:])
normal_similarity[7085, vec]  # similarity for first match

array([0.92794647, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [69]:
vec = list(normal_similarity_sorted[7085, 0:11])
normal_similarity[7085, vec]  # similarity for first match

array([1.        , 0.99678356, 0.99594064, 0.99573878, 0.99568158,
       0.99548641, 0.99543346, 0.99539609, 0.9953861 , 0.99531449,
       0.9949721 ])

In [70]:
np.mean(normal_similarity[7085, vec])

0.996012108539831

#### Check CNN Recommendations

In [71]:
cnn_recommendation_df = kaggle_df.copy(deep=True)
cnn_recommendation_df["track_id"] = cnn_recommendation_df["track_id"].astype("category")
cnn_recommendation_df["track_id"] = cnn_recommendation_df.track_id.cat.set_categories(
    cnn_recommendations[3]
)

In [72]:
cnn_track_recommendations = cnn_recommendation_df.copy(deep=True).sort_values(
    ["track_id"]
)  # sorts heirarchy
# track_recommendations.head()
cnn_track_recommendations = cnn_track_recommendations.drop_duplicates(subset="track_id")
cnn_track_recommendations.head(11)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
180270,Jazz,Green Assassin Dollar,Aware,2QbR9yeeX7XHznRZwsyh3F,34,0.8420,0.7200,73500,0.2680,0.958000,A,0.1420,-12.323,Minor,0.1020,80.132,4-Apr,0.4040
147873,Jazz,Mujo,Mirror (feat. Made in M),6qFTyyX88gmlPokNdf8h2z,44,0.5520,0.7240,94250,0.0526,0.953000,C#,0.1080,-15.380,Major,0.1520,81.605,4-Apr,0.7140
205057,Soundtrack,Marc Streitenfeld,Alpha,3tDfBzg1VZBL3MkEm6bbIp,30,0.9690,0.1020,136027,0.1500,0.950000,C,0.0688,-22.262,Major,0.0349,72.551,4-Apr,0.0620
180425,Jazz,BluntOne,Sahasrara,68huIHGDHk1kBNM0ofSUUk,36,0.4060,0.8370,96152,0.3760,0.864000,D,0.1100,-8.305,Major,0.1350,82.910,4-Apr,0.5120
89353,Hip-Hop,A$AP Twelvyy,Strapped,3U0xbYGTtYZAUv3lEwPebJ,55,0.4590,0.7240,197867,0.5340,0.000058,G#,0.1520,-8.631,Minor,0.3530,87.987,4-Apr,0.1120
147190,Jazz,Lemongrass,Sunset Glow,37R2gf2GpGrKuQffKCTzbF,49,0.7600,0.4990,279219,0.3750,0.896000,A,0.6480,-10.372,Minor,0.0281,80.095,4-Apr,0.0772
210093,World,Brian Eno,Thursday Afternoon - 2005 Digital Remaster,4t3Yh6tKkxXrc458pNI7zZ,38,0.9760,0.0918,3650688,0.0569,0.884000,C,0.0842,-31.808,Major,0.0358,81.945,4-Mar,0.0527
199145,Soundtrack,Trent Reznor,Sugar Storm,4ktkHx7JtPUk2D6ma7tUQ7,38,0.9680,0.2360,173613,0.0347,0.885000,A,0.0836,-22.849,Minor,0.0335,71.506,4-May,0.0732
220597,World,Entheogenic,Torah,1IkxzTnEP1p0IYWU21qC1y,28,0.0504,0.5460,451373,0.6060,0.721000,B,0.6540,-11.574,Minor,0.0300,133.947,4-Apr,0.0972
127720,Classical,Akira Kosemura,Grow Apart,3IQOIjxF1OfeIZ9Hg5UBtN,34,0.9610,0.4470,80880,0.2720,0.860000,F,0.1230,-21.012,Major,0.0438,75.391,4-Mar,0.1200


In [73]:
vec = list(cnn_similarity_sorted[7085, 0:11])
cnn_similarity[7085, vec]  # similarity for first match

array([1.        , 0.9990518 , 0.98749137, 0.9763511 , 0.9221045 ,
       0.9151948 , 0.89406407, 0.8691619 , 0.8667479 , 0.83266383,
       0.80967844], dtype=float32)

In [74]:
np.mean(cnn_similarity[7085, vec])

0.9156826

In [75]:
np.corrcoef(
    normal_similarity_sorted.flatten(), cnn_similarity_sorted.flatten()
)  # uncorrelated in terms of rank?

array([[ 1.00000000e+00, -7.51160957e-05],
       [-7.51160957e-05,  1.00000000e+00]])

In [76]:
normal_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6DQzmTJnaZw4A9IynCGfrM,0.446197,0.432998,0.526217,0.585114,0.453538,0.393470,0.381640,0.472515,0.441829,0.436484,0.582131,0.429712,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-28.965864,-21.389360,-19.317324,-21.354130,-21.516270,-23.200346,-24.177885,-25.294035,-24.751333,-24.149071,-26.986107,-28.732615,-28.777910,-28.232180,-28.237989,-30.091349,-29.610802,-30.583286,-26.842825,-24.891821,-29.862219,-28.541759,-27.322031,-27.285654,-25.975105,-27.822205,-30.402069,-29.787533,-25.380873,-24.870434,-28.485874,-30.992758,-30.362688,-30.110077,-31.660219,-29.567459,-27.986986,-28.580334,-29.652168,-32.246178,-33.267780,-33.264011,-33.698277,-34.058292,-31.102270,-30.543779,-32.730984,-33.274323,-34.234032,-32.236233,-30.728012,-32.439804,-34.014652,-33.142162,-31.983500,-32.185532,-33.108887,-31.718174,-32.340389,-32.058365,-32.954617,-32.621044,-32.677013,-34.328022,-35.309700,-34.261375,-32.218021,-31.745985,-31.967989,-32.379120,-32.994751,-33.861210,-34.424747,-35.061241,-35.778599,-37.029984,-38.006378,-39.420624,-39.761024,-40.701115,-42.311592,-43.039684,-43.794037,-44.971199,-45.384365,-45.927162,-47.015251,-46.453835,-47.741344,-47.351704,-46.455696,-48.026505,-48.483593,-48.324123,-48.569820,-47.866470,-47.853176,-47.425800,-47.778912,-46.980324,-47.661751,-46.500343,-47.154270,-47.186951,-48.129902,-47.701237,-48.305374,-49.650703,-50.814857,-51.338169,-52.031376,-52.891190,-53.345383,-53.385162,-54.546886,-58.247681,-69.527519,-72.046074,-72.599632,-73.240471,-73

In [77]:
cnn_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [91]:
# Initialize SpotifyOAuth
sp_oauth = SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE,
    cache_path="../.cache",
    show_dialog=False,
)


def main(normal_recommendations: List, cnn_recommendations: List):
    # Get cached token or start authorization process
    access_token_info = sp_oauth.get_cached_token()
    if access_token_info:
        access_token = access_token_info["access_token"]
    else:
        print("Authorization required. Visit the following URL to authorize:")
        auth_url = sp_oauth.get_authorize_url()
        print(auth_url)
        return

    # Initialize spotipy with access token
    sp = spotipy.Spotify(auth=access_token)

    # Replace 'Your Username' with your Spotify username
    username = USERNAME
    for track_recommendation in normal_recommendations:
        playlist_name = (
            f"Pairwise Recommendation Playlist for Track {track_recommendation[0]}"
        )
        playlist_description = f"These are the top 10 pairwise matches for track {track_recommendation[0]} by cosine similarity in descending order."

        try:
            # Create a blank playlist
            playlist = sp.user_playlist_create(
                user=username,
                name=playlist_name,
                description=playlist_description,
                public=False,
            )
            print(f"Pairwise Playlist '{playlist_name}' was created")
        except spotipy.SpotifyException as e:
            print(f"An error occurred: {e}")

        # List of track IDs (URIs) to add to the playlist
        prefix = "spotify:track:"
        track_uris = list(prefix + x for x in track_recommendation)

        # track_ids = [
        #     'spotify:track:track_id_1',
        #     'spotify:track:track_id_2',
        #     # Add more track IDs here
        # ]

        # Add tracks to the playlist
        sp.user_playlist_add_tracks(
            user=username, playlist_id=playlist["id"], tracks=track_uris
        )
    for track_recommendation in cnn_recommendations:
        playlist_name = (
            f"CNN Recommendation Playlist for Track {track_recommendation[0]}"
        )
        playlist_description = f"These are the top 10 CNN matches for track {track_recommendation[0]} by cosine similarity in descending order."

        try:
            # Create a blank playlist
            playlist = sp.user_playlist_create(
                user=username,
                name=playlist_name,
                description=playlist_description,
                public=False,
            )
            print(f"CNN Playlist '{playlist_name}' was created")
        except spotipy.SpotifyException as e:
            print(f"An error occurred: {e}")

        # List of track IDs (URIs) to add to the playlist
        prefix = "spotify:track:"
        track_uris = list(prefix + x for x in track_recommendation)

        # track_ids = [
        #     'spotify:track:track_id_1',
        #     'spotify:track:track_id_2',
        #     # Add more track IDs here
        # ]

        # Add tracks to the playlist
        sp.user_playlist_add_tracks(
            user=username, playlist_id=playlist["id"], tracks=track_uris
        )


if __name__ == "__main__":
    main(normal_recommendations, cnn_recommendations)

Pairwise Playlist 'Pairwise Recommendation Playlist for Track 0OBwxFLu6Yj61s2OagYbgY' was created
CNN Playlist 'CNN Recommendation Playlist for Track 0OBwxFLu6Yj61s2OagYbgY' was created


In [93]:
test = normal_df.copy(deep=True).reset_index()
# test.head()
test.loc[test.track_id == "2Ud3deeqLAG988pfW0Kwcl"]  # track of interest

,track_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497
2584,2Ud3deeqLAG988pfW0Kwcl,0.243606,0.279533,0.221751,0.377492,0.821773,0.322402,0.251372,0.292876,0.504456,0.726703,0.466291,0.405033,0.012553,0.011789,0.012059,0.034747,0.049094,0.030988,0.006959,0.005367,0.014339,0.04432,0.028594,0.027951,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-38.94215,-33.411198,-27.197319,-25.98834,-28.496126,-26.440634,-23.633427,-27.841377,-31.502941,-28.530811,-29.500467,-35.594536,-29.059072,-24.223267,-30.281403,-40.678936,-40.305477,-40.635609,-40.719402,-28.809755,-25.825352,-31.108908,-34.871922,-37.133602,-42.198986,-42.865707,-37.164165,-33.464024,-38.311775,-43.096985,-37.78894,-39.333748,-44.299236,-42.941689,-42.95945,-45.042435,-44.411121,-44.810745,-35.489132,-26.436434,-31.048096,-44.406891,-43.199005,-39.363285,-36.872723,-40.464821,-39.217854,-39.462654,-33.148026,-37.68293,-46.391037,-34.810688,-31.784929,-42.347023,-41.847343,-35.020813,-37.123158,-42.661163,-47.764488,-48.864964,-42.116486,-38.49205,-48.110703,-48.385899,-49.011467,-46.204517,-44.528805,-49.68465,-47.741024,-48.759232,-48.8069,-51.744366,-55.321804,-50.0173,-47.933483,-53.692467,-52.980701,-51.925297,-54.313755,-54.153793,-54.84063,-54.732479,-55.70475,-54.633133,-56.871857,-57.116005,-56.215881,-58.624977,-56.333954,-56.680477,-58.806572,-58.434856,-60.391422,-59.402897,-59.515972,-59.416206,-59.520176,-59.883743,-60.007778,-61.62899,-62.821804,-63.292274,-63.218468,-62.27903,-61.968254,-62.476582,-62.814266,-63.528908,-63.56155,-64.316391,-65.687294,-67.870659,-70.390434,-73.051033,-75.888878,-79.017975,-79.996567,-79.9972,-79.996246,-79.997475,-79.997879,-79.99839,-79.998421,-79.998444,-79.998466,-79.998482,-79.998497,-79.998512,-80.0,-80.0,-70.005447,-65.79068,-63.435143,-64.251625,-68.942642,-68.191711,-63.959343,-58.753738,-57.224442,-63.46521,-64.712952,-58.872833,-66.185089,-75.387436,-76.572647,-74.502403,-72.883865,-61.630745,-57.948853,-64.316643,-66.435501,-73.145248,-75.07093,-76.470413,-73.391251,-66.50737,-69.077499,-74.630371,-70.399063,-66.473885,-76.059875,-71.445763,-76.376999,-75.213585,-78.408554,-78.159805,-67.949287,-60.904015,-6

In [94]:
np.random.seed(123)
normal_rows = [2584]
normal_recommendations = []
dissimilar_recommendations = []
for normal_row in normal_rows:
    vec = list(normal_similarity_sorted[normal_row, 0:11])
    anti_vec = list(normal_similarity_sorted[normal_row, -10:][::-1])
    # selects vector from array
    recommendation_vec = list(normal_df.iloc[vec, :].index)
    dissimilar_vec = [recommendation_vec[0]] + list(normal_df.iloc[anti_vec, :].index)
    normal_recommendations.append(recommendation_vec)
    dissimilar_recommendations.append(dissimilar_vec)

In [95]:
normal_recommendations

[['2Ud3deeqLAG988pfW0Kwcl',
  '7LJzs1Kr8kW3d6cFTCueNV',
  '5WDlNHQW1LmcQIA15oGjiW',
  '49ihmTS9428yqABTmqQ16Y',
  '1Ly6ZKvHfosVneVTEIvMbR',
  '47W6YR93MPCGLEUReLMyDm',
  '47wtXjG0Sk6DI6cjkRBcEg',
  '3UiNahTo7xKY5qFY6Fgr0b',
  '3LWpOcr8kYmsHvqZsTLxna',
  '30vXWNZxO57z3kljfOFy02',
  '4bUnIY5kXGJKLaIO7rr3N9']]

In [98]:
np.random.seed(123)
cnn_rows = [2584]
cnn_recommendations = []
for cnn_row in cnn_rows:
    vec = list(cnn_similarity_sorted[cnn_row, 0:11])  # selects vector from array
    recommendation_vec = list(cnn_df.iloc[vec, :].index)
    cnn_recommendations.append(recommendation_vec)

In [99]:
main(normal_recommendations, cnn_recommendations)

Pairwise Playlist 'Pairwise Recommendation Playlist for Track 2Ud3deeqLAG988pfW0Kwcl' was created
CNN Playlist 'CNN Recommendation Playlist for Track 2Ud3deeqLAG988pfW0Kwcl' was created


In [236]:
vec = list(normal_similarity_sorted[10623, 0:11])
normal_similarity[10623, vec]

array([1.        , 0.99600455, 0.99565072, 0.99565072, 0.99547629,
       0.99539676, 0.99498592, 0.99497752, 0.99448599, 0.99441314,
       0.99431788])

In [237]:
vec = list(cnn_similarity_sorted[10623, 0:11])
cnn_similarity[10623, vec]

array([0.9999998 , 0.999767  , 0.99970776, 0.9995883 , 0.9995862 ,
       0.99951774, 0.99945426, 0.99944305, 0.99937904, 0.9992693 ,
       0.9991413 ], dtype=float32)

In [32]:
test2 = normal_df.copy(deep=True).reset_index()
# test.head()
test2.loc[test2.track_id == "7wobmC9Xz4stYN1ZVLWgi5"]

,track_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497
5446,7wobmC9Xz4stYN1ZVLWgi5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
kaggle_df.loc[kaggle_df.track_id.str.strip() == "2SNtwYtk9a4THKlERP0bMN"]

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
9450,Alternative,Francis and the Lights,Morning,2SNtwYtk9a4THKlERP0bMN,45,0.969,0.657,97675,0.25,0.0186,D#,0.108,-10.423,Major,0.0457,88.325,4-Apr,0.467
60708,R&B,Francis and the Lights,Morning,2SNtwYtk9a4THKlERP0bMN,49,0.969,0.657,97675,0.25,0.0186,D#,0.108,-10.423,Major,0.0457,88.325,4-Apr,0.467
99684,Children’s Music,Francis and the Lights,Morning,2SNtwYtk9a4THKlERP0bMN,49,0.969,0.657,97675,0.25,0.0186,D#,0.108,-10.423,Major,0.0457,88.325,4-Apr,0.467
